In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import PCA

from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_olivetti_faces

import lime
from lime import lime_image
from lime.wrappers.scikit_image import SegmentationAlgorithm

from skimage.color import gray2rgb, rgb2gray # since the code wants color images



## Create a simple pipeline to process the image inputs and train a classifier

In [ ]:
class PipeStep(object):
    """
    Wrapper for turning functions into pipeline transforms (no-fitting)
    """
    def __init__(self, step_func):
        self._step_func=step_func
    def fit(self,*args):
        return self
    def transform(self,X):
        return self._step_func(X)

makegray_step = PipeStep(lambda img_list: [rgb2gray(img) for img in img_list])
flatten_step = PipeStep(lambda img_list: [img.ravel() for img in img_list])

simple_rf_pipeline = Pipeline([
    ('Make Gray', makegray_step),
    ('Flatten Image', flatten_step),
    ('Normalize', Normalizer()),
    ('PCA', PCA(25)),
    ('XGBoost', GradientBoostingClassifier())
                              ])

## Get image data

In [ ]:
faces = fetch_olivetti_faces()

## Fit the model via the pipeline object

In [ ]:
# make each image color so lime_image works correctly
X_vec = np.stack([gray2rgb(iimg) for iimg in faces.data.reshape((-1, 64, 64))],0)
y_vec = faces.target.astype(np.uint8)

X_train, X_test, y_train, y_test = train_test_split(X_vec, y_vec,
                                                    train_size=0.70)
simple_rf_pipeline.fit(X_train, y_train)

## Create an explainer

In [ ]:
explainer = lime_image.LimeImageExplainer(verbose = False)
segmenter = SegmentationAlgorithm('slic', n_segments=100, compactness=1, sigma=1)

## Identify the super-pixels that drove this prediction

In [ ]:
idx = 0
num_features = 10

explanation = explainer.explain_instance(X_test[idx], 
                                         classifier_fn = simple_rf_pipeline.predict_proba, 
                                         top_labels=10, hide_color=0, num_samples=10000, segmentation_fn=segmenter)

from skimage.color import label2rgb
temp, mask = explanation.get_image_and_mask(y_test[idx], positive_only=True, num_features=num_features, hide_rest=False)
fig, (ax1) = plt.subplots(1,1, figsize = (8, 4))
ax1.imshow(label2rgb(mask,temp, bg_label = 0), interpolation = 'nearest')
ax1.set_title('Positive Regions')
temp, mask = explanation.get_image_and_mask(y_test[idx], positive_only=False, num_features=num_features, hide_rest=False)